# 데이터 전처리 작업 및 데이터 저장

이 코드는 이미지 데이터의 데이터 전처리를 수행하고, 결과를 CSV 파일로 저장하는 작업을 수행합니다.   
데이터 전처리 과정은 얼굴을 감지하고 잘라내며, 이미지를 그레이스케일로 변환하여 저장됩니다. 

In [16]:
import os
import glob

import cv2
import cvlib as cv
import numpy as np
import pandas as pd
from random import shuffle
from keras.preprocessing.image import ImageDataGenerator

from Tgram import Tgram
CROP_SIZE = (96,96) #사진크기 지정


In [17]:
class Datapreprocess:
    def __init__(self):
        """
        데이터 전처리 클래스의 초기화 메서드입니다.
        """
        self.data_lst = []  # 데이터 목록을 저장할 리스트
        self.img = None  # 이미지 데이터를 저장할 변수
        self.X_position = (0, 0)  # 얼굴 영역의 X 좌표 범위
        self.Y_position = (0, 0)  # 얼굴 영역의 Y 좌표 범위
        self.curr_emotion = ''  # 현재 감정 레이블
        self.new_file_path = ""  # 새로운 파일 경로

    def _save_crop_img(self):
        """
        얼굴 영역을 크롭하고 저장하는 내부 메서드입니다.

        :return: 크롭 및 저장 성공 여부 (True 또는 False)
        """
        try:
            img = self.img.copy()
            roi = img[
                self.Y_position[0]:self.Y_position[1],
                self.X_position[0]:self.X_position[1],
            ]
            img = cv2.resize(roi, CROP_SIZE, interpolation=cv2.INTER_CUBIC)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            self.img = img
            return True
        except:
            return False

    def _detect_face(self, img_path):
        """
        이미지에서 얼굴을 검출하는 내부 메서드입니다.

        :param img_path: 이미지 파일의 경로
        :return: 얼굴 검출 성공 여부 (True 또는 False)
        """
        try:
            self.img = cv2.imread(img_path)
            faces, _ = cv.detect_face(self.img, enable_gpu=True)
            self.X_position = faces[0][0], faces[0][2]
            self.Y_position = faces[0][1], faces[0][3]
            return True
        except:
            return False

    def _random_name(self):
        """
        임의의 파일 이름을 생성하는 내부 메서드입니다.

        :return: 생성된 임의의 파일 이름
        """
        rand_int = list(map(str, np.random.randint(low=0, high=9, size=7)))
        lst = list(map(chr, np.random.randint(low=97, high=122, size=45))) + rand_int
        shuffle(lst)
        return "".join(lst)

    def work(self, img_path, emo):
        """
        데이터 전처리 작업을 수행하는 메서드입니다.

        :param img_path: 입력 이미지 파일의 경로
        :param emo: 해당 이미지의 감정 레이블
        :return: 데이터 전처리 성공 여부 (True 또는 False)
        """
        self.curr_emotion = emo
        if self._detect_face(img_path) and self._save_crop_img():
            if self.img.shape == CROP_SIZE:
                self.new_file_path = f"./dataset/{self.curr_emotion}_{self._random_name()}.jpg"
                self.data_lst.append({"path": self.new_file_path, "label": self.curr_emotion})
                cv2.imwrite(self.new_file_path, self.img)
                return True
            else:
                return False


In [18]:
tgram = Tgram()  # Telegram 봇을 사용하여 메시지를 전송할 수 있는 Tgram 객체를 초기화합니다.
datapreprocess = Datapreprocess()  # 데이터 전처리를 수행할 Datapreprocess 클래스의 객체를 초기화합니다.
path_dir_lst = os.listdir("./pre_dataset/")  # "./pre_dataset/" 디렉토리 내에 있는 하위 디렉토리 목록을 가져옵니다.

print(path_dir_lst)  # 하위 디렉토리 목록을 출력합니다.

folder_lst = [
    f'./pre_dataset/{e}/'
    for e in path_dir_lst
]
print(folder_lst)  # 각 하위 디렉토리에 대한 경로 목록을 생성하고 출력합니다.

for emo, folder in zip(path_dir_lst, folder_lst):
    try:
        img_list = glob.glob(os.path.join(folder) + "*.jpg")  # 각 하위 디렉토리에서 .jpg 파일의 목록을 가져옵니다.
        img_list = list(map(lambda x: x.replace("\\", '/'), img_list))  # 파일 경로에서 역슬래시를 슬래시로 대체합니다.
        for img_path in img_list:
            datapreprocess.work(img_path, emo)  # 각 이미지에 대해 데이터 전처리 작업을 수행합니다.
    except Exception as e:
        tgram.fail(e)  # 예외가 발생한 경우 해당 예외를 Telegram으로 전송합니다.

pd.DataFrame(datapreprocess.data_lst).to_csv("dataset.csv", index=False)  # 데이터 전처리 결과를 CSV 파일로 저장합니다.
tgram.success()  # 데이터 전처리 및 저장이 완료되었음을 Telegram 메시지로 전송하여 성공을 알립니다.

[]
[]
